# Configure database connection

We have prepared a hosted database (triplestore / SPARQL endpoint) for you. Let’s connect to it:

In this Notebook, go to "Secrets" in the left sidebar, and add two secrets:

* Name: `sparql_user`, Value: your username (from the note)
* Name: `sparql_password`, Value: your password (from the note)

Enable "Notebook access" for both.

Then execute the following cell to set the database connection details as environment variables:

In [ ]:
import os
from google.colab import userdata
os.environ['SPARQL_USER'] = userdata.get('sparql_user')
os.environ['SPARQL_PASSWORD'] = userdata.get('sparql_password')
os.environ['SPARQL_ENDPOINT_STORE'] = "https://sd-84c02130.stardog.cloud:5820/" + userdata.get('sparql_user')
os.environ['SPARQL_ENDPOINT_QUERY'] = os.environ['SPARQL_ENDPOINT_STORE'] + "/query"
os.environ['SPARQL_ENDPOINT_UPDATE'] = os.environ['SPARQL_ENDPOINT_STORE'] + "/update"

## Test connection

To test if the database connection works, execute the following SPARQL query. It should output just the string `graph`, which means that no named graphs exist yet.

In [ ]:
!curl -X GET \
      -H "Accept:text/csv" \
      --user "${SPARQL_USER}:${SPARQL_PASSWORD}" \
      "${SPARQL_ENDPOINT_QUERY}" \
      --data-urlencode "query=SELECT DISTINCT ?graph WHERE { GRAPH ?graph { [] ?p [] . } }"

## Clear database

In case you want to **delete everything** in your database, run this SPARQL query:

In [ ]:
!curl --request "POST" \
      --header "Content-Type:application/sparql-update" \
      --data-binary "DROP ALL" \
      --user "${SPARQL_USER}:${SPARQL_PASSWORD}" \
      "${SPARQL_ENDPOINT_UPDATE}"

# Download files

In [ ]:
!git clone https://github.com/brox-it/dwt24-kg-notebook.git
%cd ./dwt24-kg-notebook/

# Install tools

## SPARQL Anything

Links: [Website](https://sparql-anything.cc/) · [Repo](https://github.com/SPARQL-Anything/sparql.anything) · [Documentation](https://sparql-anything.readthedocs.io/en/latest/)

In [ ]:
!wget https://github.com/SPARQL-Anything/sparql.anything/releases/download/0.9.0/sparql-anything-0.9.0.jar -O sparql-anything.jar

## PySHACL

Links: [Repo](https://github.com/RDFLib/pySHACL) · [PyPI](https://pypi.org/project/pyshacl/)

In [ ]:
%pip install pyshacl

# Build the knowledge graph

## 1) Understand the data (XML)

The input data is about

* persons,
* the management jobs they had, and
* the skills/competences they used in these jobs.

Each job can be assigned to a job category. Some skills have related skills.

The data is in four XML files in the `transformations_input` directory:

* `distances.xml` for skills that are related to each other
* `jobcategories.xml` for assigning job categories to jobs
* `jobcategories-skills.xml` for connecting job categories and skills
* `skills.xml` for jobs (with skills) of a person


## 2) Create the ontology (RDF)


The ontology describes what our RDF terms (e.g., classes and properties) *mean*.

The ontology itself is also formulated in RDF.

You can find  the RDF file (Turtle) in the `ontologies` directory.

### Visualize the ontology

You can generate a graph visualization using the web-app https://service.tib.eu/webvowl/.

Download the file `ontologies/o.ttl` and upload it to the web app under "Ontology" (bottom toolbar) → "Custom Ontology".

Note that this ontology just contains the terms we defined ourselves. In our RDF data, we also make use of terms that are defined in external ontologies.

## 3) Transform the data (XML to RDF)

There are many ways to convert data to RDF. One of these ways is SPARQL-based. SPARQL is the query language for RDF, but there are tools that allow querying even *non-RDF* data as if it already were RDF. One of these tools is **SPARQL Anything**.

### Transformation playground (CSV)

Here is an example CSV file:

In [ ]:
%%writefile ./transformations_playground/example.csv
id,name,color
001,Alex,blue
002,Liz,green
003,Tio,red

Here is the SPARQL `CONSTRUCT` query:

In [ ]:
%%writefile ./transformations_playground/example.sparql
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX fx:   <http://sparql.xyz/facade-x/ns/>
PREFIX xyz:  <http://sparql.xyz/facade-x/data/>
PREFIX ex:   <https://example.com/>

CONSTRUCT {

  ?s ?p ?o .

}

WHERE {

  SERVICE <x-sparql-anything:csv.headers=true,blank-nodes=true> {

    ?s ?p ?o .

  }

}

Now we call SPARQL Anything to transform the CSV to RDF (Turtle). The output gets printed here:

In [ ]:
!java -jar sparql-anything.jar \
      --query "./transformations_playground/example.sparql" \
      --configuration "location=./transformations_playground/example.csv"

### Final transformations

You can find the four query files (one for each XML file) in the `transformations` directory.

The generated RDF files (in Turtle) get saved in the `transformations_output` directory.

In [ ]:
!java -jar sparql-anything.jar \
      --query "./transformations/distances.sparql" \
      --configuration "location=./transformations_input/distances.xml" \
      --output "./transformations_output/distances.ttl" \
      --format "TTL"

In [ ]:
!java -jar sparql-anything.jar \
      --query "./transformations/jobcategories.sparql" \
      --configuration "location=./transformations_input/jobcategories.xml" \
      --output "./transformations_output/jobcategories.ttl" \
      --format "TTL"

In [ ]:
!java -jar sparql-anything.jar \
      --query "./transformations/jobcategories-skills.sparql" \
      --configuration "location=./transformations_input/jobcategories-skills.xml" \
      --output "./transformations_output/jobcategories-skills.ttl" \
      --format "TTL"

In [ ]:
!java -jar sparql-anything.jar \
      --query "./transformations/skills.sparql" \
      --configuration "location=./transformations_input/skills.xml" \
      --output "./transformations_output/skills.ttl" \
      --format "TTL"

Here we merge our four Turtle files into one, which makes it easier to validate and upload the RDF in the next steps:

In [ ]:
!cat \
  ./transformations_output/distances.ttl \
  ./transformations_output/jobcategories-skills.ttl \
  ./transformations_output/jobcategories.ttl \
  ./transformations_output/skills.ttl \
  > ./transformations_output/data.ttl

## 4) Create SHACL shapes (RDF)

[SHACL](https://www.w3.org/TR/shacl/) (Shapes Constraint Language) lets us describe what our RDF data must be *shaped* like to be considered valid/conformant.

The SHACL shapes themselves are also formulated in RDF.

You can find the RDF file (Turtle) in the `shapes` directory.



### Visualize the shapes

You can generate documentation (with an UML diagram) for the shapes using the web app [https://shacl-play.sparna.fr/play/doc](https://shacl-play.sparna.fr/play/doc).

Copy/paste the content of `shapes/shapes.ttl` and enable "Include diagram in the generated documentation".

You can display an example (`shapes/generated-doc.html`) here:

In [ ]:
import IPython
IPython.display.HTML(filename='./shapes/generated-doc.html')

### Validate the RDF

To validate our data with the SHACL shapes, we can use the tool **pySHACL**.

#### Generate RDF report

The generated report gets saved in the `shapes_output` directory.



In [ ]:
!pyshacl \
  --shacl ./shapes/shapes.ttl \
  --format turtle \
  --output ./shapes_output/report.ttl \
  ./transformations_output/data.ttl

#### Display text report

If you prefer, you can also output a report as a table here:

In [ ]:
!pyshacl \
  --shacl ./shapes/shapes.ttl \
  --format table \
  ./transformations_output/data.ttl

# Upload the knowledge graph

Thanks to the [SPARQL Protocol](https://www.w3.org/TR/sparql11-protocol/), we can upload RDF files to our triplestore using standard HTTP requests.

Here we upload not just the core dataset (which we transformed from XML), but also the ontology, the SHACL shapes, and the SHACL validation report. Each of these datasets gets uploaded into its own named graph:

* `https://graphs.brox.de/dwt24/ontologies/`
* `https://graphs.brox.de/dwt24/shacl-shapes/`
* `https://graphs.brox.de/dwt24/shacl-reports/`
* `https://graphs.brox.de/dwt24/data/`

Note that these named graphs serve just as identifiers/names, there doesn’t need to be anything at the hostname/server `graphs.brox.de`.

In [ ]:
!curl -X "PUT" \
      -H "Content-Type:text/turtle" \
      --user "${SPARQL_USER}:${SPARQL_PASSWORD}" \
      --data-binary "@./ontologies/o.ttl" \
      "${SPARQL_ENDPOINT_STORE}/?graph=https://graphs.brox.de/dwt24/ontologies/"

In [ ]:
!curl -X "PUT" \
      -H "Content-Type:text/turtle" \
      --user "${SPARQL_USER}:${SPARQL_PASSWORD}" \
      --data-binary "@./shapes/shapes.ttl" \
      "${SPARQL_ENDPOINT_STORE}/?graph=https://graphs.brox.de/dwt24/shacl-shapes/"

In [ ]:
!curl -X "PUT" \
      -H "Content-Type:text/turtle" \
      --user "${SPARQL_USER}:${SPARQL_PASSWORD}" \
      --data-binary "@./shapes_output/report.ttl" \
      "${SPARQL_ENDPOINT_STORE}/?graph=https://graphs.brox.de/dwt24/shacl-reports/"

In [ ]:
!curl -X "PUT" \
      -H "Content-Type:text/turtle" \
      --user "${SPARQL_USER}:${SPARQL_PASSWORD}" \
      --data-binary "@./transformations_output/data.ttl" \
      "${SPARQL_ENDPOINT_STORE}/?graph=https://graphs.brox.de/dwt24/data/"

# Query the knowledge graph

## Named Graphs + count of instances (per class)

In [ ]:
!curl -X GET \
      -H "Accept:text/csv" \
      --user "${SPARQL_USER}:${SPARQL_PASSWORD}" \
      "${SPARQL_ENDPOINT_QUERY}" \
      --data-urlencode "query=SELECT DISTINCT ?g ?class (COUNT(DISTINCT ?s) AS ?count) WHERE { GRAPH ?g { ?s a ?class . } } GROUP BY ?g ?class ORDER BY ?g DESC(?count)"